In [1]:
pip install torch torchvision numpy matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\wayne\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# 定義數據增強的轉換
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 將圖片大小調整為 ResNet18 的預期大小
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 使用 ImageNet 的均值和標準差進行正規化
])

# 載入數據集
dataset = datasets.ImageFolder(root='data', transform=transform)

# 劃分數據集為訓練集和測試集
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# 創建數據加載器
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=64)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)

# 加載預先訓練的 ResNet18 模型
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))  # 修改最後一層全連接層為分類層

# 將模型移動到 GPU 上進行加速
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.00001, weight_decay=0.09)

# 訓練模型
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        train_loss = 0.0
        correct = 0
        total = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}, Train')
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            progress_bar.set_postfix({'loss': train_loss / len(progress_bar), 'accuracy': 100 * correct / total})

        train_accuracy = 100 * correct / total
        val_loss, val_accuracy = evaluate_model(model, criterion, val_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

def evaluate_model(model, criterion, dataloader):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss += criterion(outputs, labels).item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = loss / len(dataloader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# 訓練模型
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

# 測試模型
test_loss, test_accuracy = evaluate_model(model, criterion, test_loader)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


C:\Users\wayne\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\wayne\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/10, Train: 100%|██████████| 22/22 [00:08<00:00,  2.68it/s, loss=0.88, accuracy=80.4] 


Epoch [1/10], Train Loss: 0.8798, Train Accuracy: 80.42%, Val Loss: 1.6342, Val Accuracy: 46.76%


Epoch 2/10, Train: 100%|██████████| 22/22 [00:08<00:00,  2.68it/s, loss=0.888, accuracy=75.8]


Epoch [2/10], Train Loss: 0.8878, Train Accuracy: 75.82%, Val Loss: 0.4518, Val Accuracy: 87.71%


Epoch 3/10, Train: 100%|██████████| 22/22 [00:08<00:00,  2.65it/s, loss=0.352, accuracy=88.8]


...

Epoch [8/10], Train Loss: 0.0723, Train Accuracy: 97.74%, Val Loss: 0.0600, Val Accuracy: 97.95%


Epoch 9/10, Train: 100%|██████████| 22/22 [00:08<00:00,  2.64it/s, loss=0.0579, accuracy=98.2]


Epoch [9/10], Train Loss: 0.0579, Train Accuracy: 98.25%, Val Loss: 0.0495, Val Accuracy: 98.63%


Epoch 10/10, Train: 100%|██████████| 22/22 [00:08<00:00,  2.66it/s, loss=0.0472, accuracy=98.6]


Epoch [10/10], Train Loss: 0.0472, Train Accuracy: 98.61%, Val Loss: 0.0355, Val Accuracy: 98.98%
Test Loss: 0.0902, Test Accuracy: 98.64%
